# Faculty Scheduler

An attempt to create a faculty scheduler, which is a much easier version of the resident scheduler.

I may try to add a database instead of excel which could be editable but simpler. I could also do CSV, but I hate Excel's complaints and difficulties with saving as CSV each time.

## variables

$W_{ftd} \subset \forall \ faculty \ \times \ \forall \ teams \ \times \ \forall \ days $

$\forall \ faculty \ \exists\ \frac{s\ shifts}{yr} := F_{FTE}*s_{FTE}$

$\forall \ faculty \ \exists\ \frac{h\ holidays}{yr}\ :=\ 1$

holidays on certain days; for 11/2025 it is 11/27.

## constraints

Exactly 1 faculty per day on each team: $\sum {faculty} \ \forall \ days \forall\ teams = 1$

## objective function
Maximize value of faculty preferences.

### micro-version - month of november with Thanksgiving

In [ ]:
import itertools as it

import numpy as np
import pandas as pd
from opre_tools import negated_bounded_span, print_full
from ortools.sat.python import cp_model

In [ ]:
faculty = pd.read_csv("faculty_scheduling_input/faculty.csv", index_col="name")
faculty

In [ ]:
days = pd.read_csv("faculty_scheduling_input/days.csv", index_col="date",parse_dates=["date"])
days.tail()

In [ ]:
days[days.holiday != "none"]

In [ ]:
teams = pd.read_csv('faculty_scheduling_input/teams.csv', index_col="name")
teams

In [ ]:
teams[teams.index.isin(["Orange","Green"])]

prototype accessor:
```python
var_grid.loc[pd.IndexSlice["Crist",'2025-11-01',"Orange"]]
```

In [ ]:
model = cp_model.CpModel()

var_grid = model.NewBoolVarSeries(
    "bool_at_", pd.Index(list(it.product(faculty.index, days.index, teams[teams.index.isin(["Orange","Green"])].index))) ## modified to exclude Purple
)


# for every day, for Orange and Green -> exactly 1 faculty on primary HS
# just realized purple is 5 days, then fold together on weekends - this raises complexity much higher. Purple excluded as above.
for d, t in it.product(days.index, teams[teams.index.isin(["Orange","Green"])].index):
    model.AddExactlyOne(var_grid.loc[pd.IndexSlice[:,d,t]])

# all work periods must be contiguous
for contig_rot, contig_rot_tail in rotations_with_contig_reqs.iterrows():
    for res in residents.index:
        hard_min = contig_rot_tail.min_contig_wks
        works = d.loc[pd.IndexSlice[res,contig_rot,:]]
        for length in range(1,hard_min):
            for start in range(len(works) - length + 1):
                model.AddBoolOr(negated_bounded_span(works,start,length))


print(model.ModelStats())

In [ ]:

# every resident's week x must have exactly 1 rotation (has not had vacations incorporated)
for n, w in it.product(residents.index, weeks.index):
    model.AddExactlyOne(d.loc[pd.IndexSlice[n, :, w]])

# # Every rotation can only go one place each week >> JUST 
# for r, w in it.product(rotations.index, weeks.index):
#     model.AddAtMostOne(d.loc[pd.IndexSlice[:, r, w]])

# apply max capacity to each rotation for each rotation:week index
for r, w in it.product(rotations.index, weeks.index):
    model.Add(sum(d.loc[pd.IndexSlice[:, r, w]]) <= rotations.resident_capacity[r]*4)

# for each rotation, set max weeks to use
for rot_name, rot_tail in rotations.iterrows():
    for res in residents.index:
        if rot_tail["max_wks"] <= MAX_WKS:  # may be
            model.Add(
                sum(d.loc[pd.IndexSlice[res, rot_name, :]]) <= rot_tail["max_wks"]
            )

# for any rotations which require contiguous weeks
rotations_with_contig_reqs = rotations[rotations.min_contig_wks > 1]
for contig_rot, contig_rot_tail in rotations_with_contig_reqs.iterrows():
    for res in residents.index:
        hard_min = contig_rot_tail.min_contig_wks
        works = d.loc[pd.IndexSlice[res,contig_rot,:]]
        for length in range(1,hard_min):
            for start in range(len(works) - length + 1):
                model.AddBoolOr(negated_bounded_span(works,start,length))

print(model.ModelStats())

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 4
solver.parameters.log_search_progress = True
solver.log_callback = print
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
    print("optimal")
    # # The status tells us if we were able to compute a solution.
    # for n, w in it.product(residents, weeks):
    #     print(
    #         "{} {}\n{}".format(
    #             n,
    #             w,
    #             solver.Values(d.loc[pd.IndexSlice[n, :, w]])[
    #                 solver.Values(d.loc[pd.IndexSlice[n, :, w]]) != 0
    #             ],
    #         )
    #     )
    print("=====Stats:======")
    print(solver.SolutionInfo())
    print(solver.ResponseStats())
elif status == cp_model.FEASIBLE:
    print("feasible")
    # # The status tells us if we were able to compute a solution.
    # for n, w in it.product(residents, weeks):
    #     print(
    #         "{} {}\n{}".format(
    #             n,
    #             w,
    #             solver.Values(d.loc[pd.IndexSlice[n, :, w]])[
    #                 solver.Values(d.loc[pd.IndexSlice[n, :, w]]) != 0
    #             ],
    #         )
    #     )
    print("=====Stats:======")
    print(solver.SolutionInfo())
    print(solver.ResponseStats())
else:
    print("no solution")

In [ ]:
print_full(solver.Values(d)[solver.Values(d) == 1].sort_index(level=(0, 2)).unstack())

In [ ]:
print(
    solver.Values(d)[solver.Values(d) == 1]
    .sort_index(level=(0, 2))
    .unstack()
    .reset_index()
    .melt(id_vars=["level_0", "level_1"])
    .query("value == 1")
    .set_index(["level_0", "variable"])
    .sort_index()["level_1"]
    .unstack()
)

In [ ]:
(
    solver.Values(d)[solver.Values(d) == 1]
    .sort_index(level=(0, 2))
    .unstack()
    .reset_index()
    .melt(id_vars=["level_0", "level_1"])
    .query("value == 1")
    .set_index(["level_0", "variable"])
    .sort_index()["level_1"]
    .unstack()
    .to_excel("output_schedule.xlsx")
)